In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [5]:
classSearchAPI = 'https://classes.colorado.edu/api/?page=fose&route=search&camp=BLDR'
classDetailsAPI = 'https://classes.colorado.edu/api/?page=fose&route=details'
dataDir = 'data/'
rawClassDataFilename = dataDir+'classes.json'
classDetailsDataFilename = dataDir+'classDetails.json'
maxThreads = 30

In [16]:
# Fetch classes data - limit to boulder campus only
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Sec-Fetch-Site': 'same-origin',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Sec-Fetch-Mode': 'cors',
    'Host': 'classes.colorado.edu',
    'Origin': 'https://classes.colorado.edu',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2.1 Safari/605.1.15',
    'Referer': 'https://classes.colorado.edu',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'empty',
    'X-Requested-With': 'XMLHttpRequest',
}
body = '{"other":{"srcdb":"2247"},"criteria":[{"field":"camp","value":"BLDR"}]}'

response = requests.post(classSearchAPI, headers=headers, data=body)

with open(rawClassDataFilename, 'w') as file:
    file.write(response.text)

In [17]:
def getClassDetails(group, crns):
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2.1 Safari/605.1.15',
        'X-Requested-With': 'XMLHttpRequest',
    }

    data = json.dumps({
        "group": "code:" + group,
        "key": "",
        "srcdb": "2247",
        "matched": "crn:" + ",".join(crns) # Different sections
    })
    
    response = requests.post(classDetailsAPI, headers=headers, data=data)
    return response.json()

In [18]:
# Given the raw classes data, populate class details

data = json.load(open(rawClassDataFilename))
df = pd.DataFrame(data['results'])

# Group by code and collect its crns
groupByCode = df.groupby('code')['crn'].apply(list).reset_index()
details_results = []

# With multithreading, fetch class details concurrently
with ThreadPoolExecutor(max_workers=maxThreads) as executor:
    futures = [executor.submit(getClassDetails, row['code'], row['crn']) for index, row in groupByCode.iterrows()]
    details_results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures))]

# Save class details to a JSON file
json_data = json.dumps(details_results, indent=4)
with open(classDetailsDataFilename, 'w') as file:
    file.write(json_data)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3244/3244 [00:50<00:00, 63.93it/s]
